In [1]:
import cv2
import pickle
import pandas as pd
import mediapipe as mp
from keras_facenet import FaceNet
from keras.models import load_model

In [2]:
def load_pickle_model(filename):
    with open(filename, 'rb') as file:
        model=pickle.load(file)
    return model

In [3]:
face_detection=mp.solutions.face_detection.FaceDetection()
facenet_embedder=FaceNet()

In [4]:
face_recognition_model=load_model('models/face_recognition.keras')
label_encoder=load_pickle_model('models/label_encoder.pickle')

In [5]:
def detect_faces(frame):
    height, width, _=frame.shape
    results=face_detection.process(frame)

    detected_faces=[]
    if results.detections:
        for detection in results.detections:
            face_bounding=detection.location_data.relative_bounding_box
            x=int(face_bounding.xmin * width)
            y=int(face_bounding.ymin * height)
            w=int(face_bounding.width * width)
            h=int(face_bounding.height * height)
            detected_faces.append((x,y,w,h))

    return detected_faces

In [6]:
def predict_face_names(image):
    embedded_image=facenet_embedder.embeddings([image])
    pred_prob=face_recognition_model.predict(embedded_image)
    ypred=pred_prob.argmax(axis=1)
    prob=pred_prob[0, ypred[0]]
    return label_encoder.inverse_transform(ypred)[0], int(round(prob,2)*100)

In [7]:
def show_faces_recognised(frame, face_list, probab_list):
    rgb_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detected_faces=detect_faces(rgb_frame)
    for face in detected_faces:
        x, y, w, h=face
        cropped_face=rgb_frame[y:y+h, x:x+w]
        predicted_name, prob=predict_face_names(cropped_face)
        face_list.append(predicted_name)
        probab_list.append(prob)
        frame=cv2.rectangle(frame, (x, y), (x+w, y+h), (0,165,255))
        frame=cv2.putText(frame, f"{predicted_name} ({prob}%)", (x, y), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1.2, color=(0,0,0))
    return frame

In [ ]:
frame_count=0
frame_skip=7
faces_list=[]
probab_list=[]
capture=cv2.VideoCapture("dataset/video_Big B and SRK.mp4")
while(True):
    ret, frame=capture.read()
    frame=cv2.resize(frame, (640,400))
    if frame_count%frame_skip != 0:
        frame=show_faces_recognised(frame, faces_list, probab_list)
    cv2.imshow("Face Detection", frame)
    frame_count+=1
    if(cv2.waitKey(1) & 0xff==ord('q')):
        break

capture.release()
cv2.destroyAllWindows()
print(frame_count, frame_skip)

In [10]:
recognised_data=pd.DataFrame({'name': faces_list, 'probab': probab_list})
recognised_data.groupby('name').mean()

,probab
name,
Amitabh Bachchan,100.0
Shah Rukh Khan,100.0
